# Equivilent Width

This notebook compares pseudo-equivilent widths (pW) between CMFGEN models and CSP spectroscopy. pW values are determine according to the prescription set forth in the CSP DR1 data release Folatelli et al. 2013. For each feature, "the pW is obtained by defining a straight 'continuum' level between two neighboring flux peaks and computing the integral of the spectrum flux relative to the continuum" (Folatelli 2013).


In [1]:
import sys

import sncosmo
from astropy.table import Table, join
from matplotlib import pyplot as plt
from sndata.csp import dr1
from specutils.analysis import equivalent_width
from specutils.fitting import fit_generic_continuum

sys.path.insert(0, '../')
from analysis import equivalent_width as ew
from analysis import models
from analysis import utils


In [2]:
models.register_sources(force=True)
dr1.download_module_data()

# Load models for different masses
m102 = sncosmo.Model(sncosmo.get_source('CMFGEN', version=1.02))
m104 = sncosmo.Model(sncosmo.get_source('CMFGEN', version=1.04))
m14 = sncosmo.Model(sncosmo.get_source('CMFGEN', version=1.4))
m17 = sncosmo.Model(sncosmo.get_source('CMFGEN', version=1.7))

# Add an extinction component to each model
# The extinction component is zero until changed
model_list = [m102, m104, m14, m17]
for model in model_list:
    model.add_effect(sncosmo.F99Dust(), 'ext', 'rest')


In [3]:
def plot_spectra(obs_dates, wavelength, flux, **kwargs):
    """Plot spectra
    
    Args:
        obs_dates  (list): A list of observed MJD dates for each spectra
        wavelength (list): A 2d list of wavelength values for each date
        flux       (list): A 2d list of flux values for each date
        Any other arguments for ``axis.plot``
    """
    
    fig, axis = plt.subplots(1, 1, figsize=(10, 10))    
    plot_data = zip(obs_dates, wavelength, flux)
    for i, (date, wave, flux) in enumerate(plot_data):
        axis.plot(wave, flux + i * 2e-15, label=str(date), **kwargs)
        axis.legend()
    
    return fig, axis


In [4]:
demo_id = '2004ef'
demo_data = dr1.get_data_for_id(demo_id)
obs_dates, wavelength, flux = utils.parse_spectra_table(demo_data)
# plot_spectra(obs_dates, wavelength, flux)


## Fitting the Continuum

We consider the same set of features considered in Folatelli et al. 2013.

In [5]:
ew.features

{'pW1': {'feature_id': 'Ca\u2009ii H&K',
  'lower_blue': 3500,
  'upper_blue': 3800,
  'lower_red': 3900,
  'upper_red': 4100},
 'pW2': {'feature_id': 'Si\u2009ii λ4130',
  'lower_blue': 3900,
  'upper_blue': 4000,
  'lower_red': 4000,
  'upper_red': 4150},
 'pW3': {'feature_id': 'Mg\u2009ii, Fe\u2009ii',
  'lower_blue': 3900,
  'upper_blue': 4150,
  'lower_red': 4450,
  'upper_red': 4700},
 'pW4': {'feature_id': 'Fe\u2009ii, Si\u2009ii',
  'lower_blue': 4500,
  'upper_blue': 4700,
  'lower_red': 5050,
  'upper_red': 5550},
 'pW5': {'feature_id': 'S\u2009ii λ5449, λ5622',
  'lower_blue': 5150,
  'upper_blue': 5300,
  'lower_red': 5500,
  'upper_red': 5700},
 'pW6': {'feature_id': 'Si\u2009ii λ5972',
  'lower_blue': 5550,
  'upper_blue': 5700,
  'lower_red': 5800,
  'upper_red': 6000},
 'pW7': {'feature_id': 'Si\u2009ii λ6355',
  'lower_blue': 5800,
  'upper_blue': 6000,
  'lower_red': 6200,
  'upper_red': 6600},
 'pW8': {'feature_id': 'Ca\u2009ii IR triplet',
  'lower_blue': 7500,
  'u

In [6]:
def plot_ew(obs_dates, wavelength, flux, feature_table, **kwargs):
    """
    
    Args:
        obs_dates      (list): A list of observed MJD dates for each spectra
        wavelength     (list): A 2d list of wavelength values for each date
        flux           (list): A 2d list of flux values for each date
        feature_table (Table): A table defining spectral features
        Any other arguments for ``axis.plot``
    """

    fig, axis = plot_spectra(obs_dates, wavelength, flux)

    for i, (time, wave, fluxx) in enumerate(zip(obs_dates, wavelength, flux)):
        for feature in ew.feature_table:
            try:
                feat_wave, feat_flux = ew.get_feature_coordinates(wave, fluxx, feature)

            except ew.UnobservedFeature:
                continue

            # Get observed wavelength and flux for each feature
            indices = (feat_wave[0] < wave) & (wave < feat_wave[1])
            spectral_wave = wave[indices]
            spectral_flux = fluxx[indices]
            
            # Get continuum and plot
            cont_func = ew.get_continuum_func(*feat_wave, *feat_flux)
            continuum_flux = cont_func(spectral_wave)
            axis.fill_between(
                spectral_wave,
                spectral_flux + i * 2e-15,
                continuum_flux + i * 2e-15, 
                color='grey', 
                alpha=.75)
            

In [7]:
# plot_ew(obs_dates, wavelength, flux, ew.features)

## Comparing CMFGEN Against Observations

In [8]:
model_list = (m102, m104, m14 , m17)
ew_results = ew.tabulate_pew_spectrum(obs_dates[0], wavelength[0], flux[0], model_list, False)
ew_results


model,version,pW2,pW2_start,pW2_end,pW3,pW3_start,pW3_end,pW4,pW4_start,pW4_end,pW5,pW5_start,pW5_end,pW6,pW6_start,pW6_end,pW7,pW7_start,pW7_end
str100,str100,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
obs,,52.225438367924625,3917.0316,4073.6855,184.80783257869825,3917.0316,4454.4748,205.25239219537804,4526.7766,5066.6299,102.15214197976803,5163.0322,5606.4831,41.9588278788157,5606.4831,5833.0287,196.28587634612143,5833.0287,6356.0115
CMFGEN,CMFGEN,-0.00144618085945325,3996.5636,4001.3837,448.48345400492724,3996.5636,4688.2506,237.59471225406696,4688.2506,5230.5139,36.78667065268979,5230.5139,5500.4405,172.20793988023706,5551.0518,5873.9997,230.10608198799537,5873.9997,6515.0754
CMFGEN,CMFGEN,0.0,3998.9737,4001.3837,462.7805807243851,3998.9737,4693.0707,112.14627762749322,4693.0707,5052.1695,29.58315582811659,5235.334,5500.4405,167.35163002350123,5551.0518,5873.9997,269.6995374519453,5873.9997,6517.4855
CMFGEN,CMFGEN,0.15730971099052482,3998.9737,4035.1245,386.58239905057263,4035.1245,4681.0204,242.99896436813532,4681.0204,5225.6938,36.30588457177785,5225.6938,5500.4405,172.64233076376905,5551.0518,5873.9997,212.71121405423793,5873.9997,6512.6654
CMFGEN,CMFGEN,-0.009105108669825057,3991.7435,4001.3837,494.416445154679,3991.7435,4683.4305,121.11719265998437,4683.4305,5052.1695,44.451503110807494,5249.7944,5500.4405,176.62597043060885,5551.0518,5886.05,311.7512019569669,5886.05,6551.2263


In [9]:
from sndata.csp import dr1
# ew.tabulate_pew(dr1, model_list, True, verbose=True)

In [10]:
for source in sources:
    plt.plot(ew_results['time'], ew_results[f'pW7_C{source.version}'], label=source.version)
    plt.scatter(ew_results['time'], ew_results[f'pW7_C{source.version}'], label=None)

plt.plot(ew_results['time'], ew_results[f'pW7'], label='data')
plt.scatter(ew_results['time'], ew_results[f'pW7'], label=None)
plt.legend()


NameError: name 'sources' is not defined

In [13]:
m104.param_names

['z', 't0', 'x0', 'extebv']